## Projeto de ciência de dados

In [7]:
import pandas as pd

In [3]:
URL = "https://raw.githubusercontent.com/natanaelmartins/data-science/main/brazil_covid19.csv"

In [4]:
dataset = pd.read_csv(URL)

In [11]:
dataset.head()

,date,region,state,cases,deaths
0,2020-02-25,Centro-Oeste,DF,0.0,0
1,2020-02-25,Centro-Oeste,GO,0.0,0
2,2020-02-25,Centro-Oeste,MS,0.0,0
3,2020-02-25,Centro-Oeste,MT,0.0,0
4,2020-02-25,Nordeste,AL,0.0,0
